<h1>Data Cleaning</h1>

# Imports

In [15]:
import sys
from pathlib import Path
from datetime import datetime

import pandas as pd

sys.path.insert(0, r"C:\Users\vynde\PycharmProjects\dataanalysis")
from databridger import Database

# Issue Tracking

In [61]:
# TODO: export class to module and import it

class IssueTracker:
    VALID_FIELDS = ["description", "resolution", "severity", "potential_cause", "relevant_data", "notes"]

    def __init__(self):
        self.df = pd.DataFrame(columns=["issue_id", "version", "status", *self.VALID_FIELDS])
        self.issue_count = 0
    
    def __repr__(self):
        return self.df.__repr__()

    def _repr_html_(self):
        return self.df._repr_html_()

    def show(self):
        return self.df.style.format({
            'description': lambda x: str(x).replace("\n", "<br>"),
            'notes': lambda x: str(x).replace("\n", "<br>")
        })
            
    def new_issue(self, description, severity, potential_cause=None, relevant_data=None, notes=None):
        issue = {
            "issue_id": self.issue_count + 1,
            "version": 1, 
            "status": "Open",
            "description": description,
            "resolution": None,
            "severity": severity,
            "potential_cause": potential_cause,
            "relevant_data": relevant_data,
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "notes": notes
        }
        self.df = pd.concat([self.df, pd.DataFrame([issue])], ignore_index=True)
        
        self.issue_count += 1

    def update_issue(self, /, status=None, description=None, severity=None, potential_cause=None, relevant_data=None, notes=None, issue_id=None, resolution=None):

        if issue_id is None:
            issue = self.df.iloc[-1]
        else:
            issue = self.df.loc[self.df["issue_id"]==issue_id, :].iloc[-1]

        new_issue = issue.copy()
        print(new_issue.shape)
        new_issue["version"] = issue["version"] + 1

        if (new_issue["status"] == "resolved") and (issue["issue_id"] == new_issue["issue_id"]):
            raise Exception("Issue already resolved.")
        
        # Apply the updates
        if status is not None:
            new_issue["status"] = status
        if description is not None:
            new_issue["description"] = description
        if severity is not None:
            new_issue["severity"] = severity
        if potential_cause is not None:
            new_issue["potential_cause"] = potential_cause
        if relevant_data is not None:
            new_issue["relevant_data"] = relevant_data
        if resolution is not None:
            new_issue["resolution"] = resolution
        
        # overwrite notes always
        new_issue["notes"] = notes
        
        self.df = pd.concat([self.df, pd.DataFrame([new_issue])])

    def resolve_issue(self, resolution, issue_id=None):
        if not issue_id:
            issue_id = self.issue_count

        self.update_issue(status="resolved", resolution=resolution, issue_id=issue_id)

    def export_to_csv(self, filename="issues.csv"):
        self.df.to_csv(filename, index=False)

# create instance
issue_tracker = IssueTracker()

# Resources

In [17]:
# folders
raw_data_folder = Path('..') / 'data' / 'raw'
report_folder = Path('..') / 'reports'

# output files
database_summary_file = report_folder / "database_summary.xlsx"
overlap_ratio_file = report_folder / "column_overlap_ratios.xlsx"

---

# Load Data

New method for loading file based data base

In [18]:
db_raw = Database("csv", raw_data_folder)
db_raw.table_mapping

{'customers': ['orders'],
 'order_items': ['orders',
  'order_payments',
  'order_reviews',
  'products',
  'sellers'],
 'order_payments': ['orders', 'order_items', 'order_reviews'],
 'order_reviews': ['orders', 'order_items', 'order_payments'],
 'orders': ['order_items', 'order_payments', 'order_reviews', 'customers'],
 'product_category_name_translation': ['products'],
 'products': ['order_items', 'product_category_name_translation'],
 'sellers': ['order_items']}

Inspect file names

In [19]:
for csv_file in raw_data_folder.glob('*.csv'):
    print(csv_file)

..\data\raw\olist_customers_dataset.csv
..\data\raw\olist_geolocation_dataset.csv
..\data\raw\olist_orders_dataset.csv
..\data\raw\olist_order_items_dataset.csv
..\data\raw\olist_order_payments_dataset.csv
..\data\raw\olist_order_reviews_dataset.csv
..\data\raw\olist_products_dataset.csv
..\data\raw\olist_sellers_dataset.csv
..\data\raw\product_category_name_translation.csv


Load all data sets into a database (dictionary of dataframes)

In [20]:
db = dict()  # database
for csv_file in raw_data_folder.glob('*.csv'):
    dataset_name = str(csv_file.stem).replace("olist_", "").replace("_dataset", "")
    db[dataset_name] = pd.read_csv(csv_file)

db.keys()

dict_keys(['customers', 'geolocation', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'product_category_name_translation'])

Convert to datetime matching this format yyyy-mm-dd HH:MM:SS

In [21]:
for key in db:
    for column in db[key].columns:
        if db[key][column].dtype == 'object':
            if all(db[key][column].str.match(r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}")):
                db[key][column] = pd.to_datetime(db[key][column])
                print(f"converted:   {key:15} / {column:30} from object to {db[key][column].dtype}")

# also tried another version with try except trying to convert every object-type column to datetime
# showed a very strange behavior and messed up all types in the dataframe


converted:   orders          / order_purchase_timestamp       from object to datetime64[ns]
converted:   orders          / order_approved_at              from object to datetime64[ns]
converted:   orders          / order_delivered_carrier_date   from object to datetime64[ns]
converted:   orders          / order_delivered_customer_date  from object to datetime64[ns]
converted:   orders          / order_estimated_delivery_date  from object to datetime64[ns]
converted:   order_items     / shipping_limit_date            from object to datetime64[ns]
converted:   order_reviews   / review_creation_date           from object to datetime64[ns]
converted:   order_reviews   / review_answer_timestamp        from object to datetime64[ns]


> **Summary**: 
>- Loaded 9 CSV files into a dictionary, effectively creating a database.
>- Identified 8 columns as datetime columns

---

# Inspect Data

In [22]:
for name, df in db.items():
    print(f"{name}: {df.shape}")

customers: (99441, 5)
geolocation: (1000163, 5)
orders: (99441, 8)
order_items: (112650, 7)
order_payments: (103886, 5)
order_reviews: (99224, 7)
products: (32951, 9)
sellers: (3095, 4)
product_category_name_translation: (71, 2)


## Database Column Characterization & Classification

In [23]:
tables = [key for key in db for column in db[key]]
columns = [column for key in db for column in db[key]]
summary = []

for table, column in zip(tables, columns):
    series = db[table][column]
    data = {"table": table, "column": column}
    
    # Common metrics for all types
    data["count"] = len(series)
    unique_count = series.nunique()
    data["unique_count"] = unique_count

    # Determine if the column is a key
    if (unique_count == len(series)) \
    or (column.endswith("_id")):
        data["duplicated_count"] = len(series) - series.drop_duplicates().size
        data["type"] = "key"
    # Determine if the column is temporal
    elif series.dtype in ['datetime64[ns]', 'datetime64[ns, tz]']:
        data["min_date"] = series.min()
        data["max_date"] = series.max()
        data["range"] = series.max() - series.min()
        data["type"] = "temporal"
    # Determine if the column is numeric
    elif series.dtype in ['int64', 'float64']:
        data["min"] = series.min()
        data["max"] = series.max()
        data["mean"] = series.mean()
        data["type"] = "numeric"
    # Determine if the column is nominal (e.g. more than 10 unique text values)
    elif series.dtype == 'object' and unique_count > 10:
        mode_data = series.mode()
        data["mode"] = mode_data[0] if not mode_data.empty else None
        data["mode_count"] = (series == data["mode"]).sum()
        data["type"] = "nominal"
    # Determine if the column is categorical (e.g. 10 or fewer unique text values)
    elif series.dtype == 'object' and unique_count <= 10:
        mode_data = series.mode()
        data["mode"] = mode_data[0] if not mode_data.empty else None
        data["mode_count"] = (series == data["mode"]).sum()
        data["type"] = "categorical"
    else:
        data["type"] = "unknown"
    
    # Append the computed metrics for the column to the summary list
    summary.append(data)

# Compile the summary list into a DataFrame for a neat presentation
df_summary = pd.DataFrame(summary)

# Split the dataframes by type and drop nan columns
df_summaries = {name: df.dropna(axis=1).drop(columns="type") for name ,df in df_summary.groupby("type")}

In [24]:
df_summary

,table,column,count,unique_count,duplicated_count,type,min,max,mean,mode,mode_count,min_date,max_date,range
0,customers,customer_id,99441,99441,0.0,key,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,customers,customer_unique_id,99441,96096,3345.0,key,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,customers,customer_zip_code_prefix,99441,14994,NaN,numeric,1003.000000,99990.000000,35137.474583,NaN,NaN,NaT,NaT,NaT
3,customers,customer_city,99441,4119,NaN,nominal,NaN,NaN,NaN,sao paulo,15540.0,NaT,NaT,NaT
4,customers,customer_state,99441,27,NaN,nominal,NaN,NaN,NaN,SP,41746.0,NaT,NaT,NaT
5,geolocation,geolocation_zip_code_prefix,1000163,19015,NaN,numeric,1001.000000,99990.000000,36574.166466,NaN,NaN,NaT,NaT,NaT
6,geolocation,geolocation_lat,1000163,717360,NaN,numeric,-36.605374,45.065933,-21.176153,NaN,NaN,NaT,NaT,NaT
7,geolocation,geolocation_lng,1000163,717613,NaN,numeric,-101.466766,121.105394,-46.390541,NaN,NaN,NaT,NaT,NaT
8,geolocation,geolocation_city,1000163,8011,NaN,nominal,NaN,NaN,NaN,sao paulo,135800.0,NaT,NaT,NaT
9,geolocation,geolocation_state,1000163,27,NaN,nominal,NaN,NaN,NaN,SP,404268.0,NaT,NaT,NaT


## Inspect Key Columns

In [25]:
df_summaries["key"].sort_values(by="column")

,table,column,count,unique_count,duplicated_count
0,customers,customer_id,99441,99441,0.0
11,orders,customer_id,99441,99441,0.0
1,customers,customer_unique_id,99441,96096,3345.0
10,orders,order_id,99441,99441,0.0
18,order_items,order_id,112650,98666,13984.0
25,order_payments,order_id,103886,99440,4446.0
31,order_reviews,order_id,99224,98673,551.0
19,order_items,order_item_id,112650,21,112629.0
50,product_category_name_translation,product_category_name,71,71,0.0
51,product_category_name_translation,product_category_name_english,71,71,0.0


In [62]:
issue_tracker.new_issue(
    description="customer_id is unique in customers and orders table",
    severity="minor",
    potential_cause=None,
    relevant_data="customers -> customer_id | orders -> customer_id",
    notes="""one-to-one relationship; 
    tables could be merged; 
    each customer has done exactly one order; 
    structure intenional?; 
    check if cutomer locations if they are also different for all customers"""
)

In [63]:
issue_tracker.new_issue(
    description="key column with duplicated values",
    severity="minor",
    potential_cause="surrogate key",
    relevant_data="customers -> customer_unique_id",
    notes="""Usability needs to be checked."""
)

In [67]:
issue_tracker.show()

,issue_id,version,status,description,resolution,severity,potential_cause,relevant_data,notes,timestamp
0,1,1,Open,customer_id is unique in customers and orders table,None,minor,None,customers -> customer_id | orders -> customer_id,one-to-one relationship; tables could be merged; each customer has done exactly one order; structure intenional?; check if cutomer locations if they are also different for all customers,2023-08-11 17:32:55
1,2,1,Open,key column with duplicated values,None,minor,surrogate key,customers -> customer_unique_id,Usability needs to be checked.,2023-08-11 17:32:56


In [ ]:
db["customers"][db["customers"]["customer_unique_id"]=="8d50f5eadf50201ccdcedfb9e2ac8455"]

## Inspect Nominal Columns

In [ ]:
df_summaries["nominal"]

## Inspect Categorical Columns

In [ ]:
df_summaries["categorical"]

## Inspect Numeric Columns

In [ ]:
df_summaries["numeric"]

## Inspect Temporal Columns

In [ ]:
df_summaries["temporal"]

## Analysis of Potential Foreign Key Relationships

Matrix of matching_ratios of all columns

In [ ]:
from tqdm.notebook import tqdm

def compute_match_ratio(from_column, to_column):
    from_set = set(from_column.dropna())
    to_set = set(to_column.dropna())

    # Number of matching entries
    matching_entries = len(from_set.intersection(to_set))

    # Total unique entries in both columns
    total_entries = len(from_set.union(to_set))

    return matching_entries / total_entries if total_entries != 0 else 0

# Iterate over each table and column
db_columns = [(table, column) for table in db.keys() for column in db[table].columns]

data = {}

# Creating a single progress bar for inner loops
inner_pbar = tqdm(total=len(db_columns), desc='Inner Loop', unit='column', leave=False)

# Outer loop with its own progress bar
for tab_a, col_a in tqdm(db_columns, desc='Outer Loop', unit='column'):
    
    # Reset the inner progress bar after each iteration of the outer loop
    inner_pbar.n = 0
    inner_pbar.desc = f"{tab_a}/{col_a}"
    inner_pbar.last_print_n = 0
    inner_pbar.refresh()
    
    for tab_b, col_b in db_columns:
        key = (tab_a, col_a)
        if key not in data:
            data[key] = []
        data[key].append(compute_match_ratio(db[tab_a][col_a], db[tab_b][col_b]))
        
        # Update the inner progress bar
        inner_pbar.update(1)

df = pd.DataFrame(data, index=db_columns)
df


Export matching_ratio matrix to Excel

In [ ]:
df.to_excel(overlap_ratio_file)

## Analysis of Missing Value Relationships Across Tables

Quantifying inter-table key relationships and overlaps

In [ ]:
# Initialize the lists to store the table/key mappings
from_tables = []
from_columns = []
to_tables = []
to_columns = []

# Iterate over each table and column
for key in db:
    for column in db[key].columns:
        # If the column ends with "_id", it's potentially a foreign key
        if column.endswith("_id"):
            # Search for potential primary keys in other tables
            for potential_key in db:
                if column in db[potential_key].columns and key != potential_key:
                    from_tables.append(key)
                    from_columns.append(column)
                    to_tables.append(potential_key)
                    to_columns.append(column)

# Construct the mapping dataframe
df_mapping = pd.DataFrame({
    'from_table': from_tables,
    'from_column': from_columns,
    'to_table': to_tables,
    'to_column': to_columns
})

###
### the logic above is already implemented in Database.column_mapping
##

def is_subset(row):
    """Check if one column's unique values are a subset of the other column's unique values."""
    from_set = set(db[row["from_table"]][row["from_column"]].dropna())
    to_set = set(db[row["to_table"]][row["to_column"]].dropna())
    return from_set.issubset(to_set)

df_mapping["is_subset"] = df_mapping.apply(is_subset, axis=1)

def subset_ratio(row):
    """Compute the ratio of unique values from 'from_column' found in 'to_column'."""
    from_set = set(db[row["from_table"]][row["from_column"]].dropna())
    to_set = set(db[row["to_table"]][row["to_column"]].dropna())
    return len(from_set.intersection(to_set)) / len(from_set) if len(from_set) != 0 else 0

df_mapping["subset_ratio"] = df_mapping.apply(subset_ratio, axis=1)

def shared_value_ratio(row):
    """Compute the ratio of shared unique values between two columns."""
    from_set = set(db[row["from_table"]][row["from_column"]].dropna())
    to_set = set(db[row["to_table"]][row["to_column"]].dropna())

    # Number of matching entries
    matching_entries = len(from_set.intersection(to_set))

    # Total unique entries in both columns
    total_entries = len(from_set.union(to_set))

    return matching_entries / total_entries if total_entries != 0 else 0

df_mapping["shared_value_ratio"] = df_mapping.apply(shared_value_ratio, axis=1)

def missing_count(row):
    from_set = set(db[row["from_table"]][row["from_column"]].dropna())
    to_set = set(db[row["to_table"]][row["to_column"]].dropna())
    
    # Missing count
    return len(from_set.difference(to_set))

df_mapping["missing_count"] = df_mapping.apply(missing_count, axis=1)

df_mapping.sort_values(by="missing_count", ascending=True)

